<a href="https://colab.research.google.com/github/fullpistoladev/relatorio_parcial/blob/main/Pibic_jaccard_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!python -m spacy download pt_core_news_lg

2023-05-22 13:24:52.876625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 13:24:54.847412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [10]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('floresta')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Unzipping corpora/floresta.zip.


True

In [6]:
import spacy
nlp = spacy.load('pt_core_news_lg') 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

path_questions = "drive/MyDrive/Colab Notebooks/data/questions.xlsx"
path_answers = "drive/MyDrive/Colab Notebooks/data/answers.xlsx"

questions = pd.read_excel(path_questions)
answers = pd.read_excel(path_answers)
print(questions, answers)

                        questaoMatrizId  \
0  0b991be1-84ab-40af-8d4b-94d4aa0ac9b2   
1  83099bc5-0c76-4633-a6f2-f89d50c673aa   
2  e39bcb05-5a01-4b72-81ca-049b438fce69   
3  bc490271-4f7e-465a-8e8e-c1d1a5936612   

                                           enunciado  \
0  Sobre Orientação a Objetos, descreva o que voc...   
1  Descreva o que você entende por classes e obje...   
2  Herança é um dos dos pilares da Programação Or...   
3  Os atributos e métodos em Java podem receber m...   

                    discursiva.expectativaDeResposta  
0  Polimorfismo é uma característica das linguage...  
1  Classe são modelos descritivos para a criação ...  
2  Herança é um conceito que permite que uma clas...  
3  private -&gt; Permite acesso apenas dentro da ...                             questaoMatrizId                  nomeCandidato  \
0    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2      ANDREON SOUZA DE MEDEIROS   
1    83099bc5-0c76-4633-a6f2-f89d50c673aa      ANDREON SOUZA DE MEDEIROS   


In [61]:

def to_lowercase(data):
    return [line.replace(line, line.lower()) for line in data]

def remove_stopwords(data):
    stop_words = spacy.lang.pt.stop_words.STOP_WORDS
    filtered_tokens = []
    
    for line in data:
        # tokenize the line with spaCy
        doc = nlp(line)
        
        # remove stopwords from the doc
        filtered_tokens_line = [token.text for token in doc if token.text.lower() not in stop_words]
        
        # join the remaining tokens back into a single string
        filtered_line = ' '.join(filtered_tokens_line)
        
        filtered_tokens.append(filtered_line)

    return filtered_tokens

def remove_punct(data):
    #regex para tudo que não for [A-Za-Z0-9] e espaçamento 
    return [re.sub(r'[^\w\s]', ' ', str(line)) for line in data]


def lemmatize(data):
    lemmatizer = WordNetLemmatizer()
    lem_tokens = []
    for line in data:
        tokens = word_tokenize(line, language='portuguese')
        lem_line = []
        for token in tokens:
            lem_token = lemmatizer.lemmatize(token)
            lem_line.append(lem_token)
        lem_string = ' '.join(lem_line)
        lem_tokens.append(lem_string)
    return lem_tokens

def stem(data):
    stemmer = SnowballStemmer('portuguese')
    stem_tokens = []
    for line in data:
        tokens = word_tokenize(line, language='portuguese')
        stem_line = []
        for token in tokens:
            stem_token = stemmer.stem(token)
            stem_line.append(stem_token)
            #print(stem_line)
        #stem_string = ' '.join(stem_line)
        #stem_tokens.append(stem_string)
    return stem_line

def preprocess_text(data):
    
    data = remove_punct(data)
    data = to_lowercase(data)
    #data = remove_stopwords(data)
    #data = lemmatize(data)
    data = stem(data)

    return data

In [57]:
teste = ["Isso é um teste acredite no seu potencial seu bosta kkkkk"]
print(teste)
teste = preprocess_text(teste)
print(teste)

['Isso é um teste acredite no seu potencial seu bosta kkkkk']
['test', 'acredit', 'potencial', 'bost', 'kkkkk']


In [58]:
def preprocess(sentence):
    result = []
    tokens = nlp(sentence)
    for token in tokens:
        # remove determiners and punctuation or spaces 
        if token.pos_ in ['PUNCT','SPACE', 'DET']:
            continue
        else:
            # the lemma of pronouns is stored as '-PRON-' in spacy; we want the actual pronoun
            if token.pos_ == 'PRON':
                result.append(token.text.lower())
            else:
                result.append(token.lemma_)
    return result

In [59]:
def jaccard_coefficient(sent1, sent2):
    tokens1 = preprocess_text(sent1)
    tokens2 = preprocess_text(sent2)
    set1 = set(tokens1)
    set2 = set(tokens2)
    print(tokens1, tokens2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

In [62]:
jaccard_similarity = pd.DataFrame()

for questionId in range(4):

  temp = answers[answers['questaoMatrizId'] ==  questions['questaoMatrizId'][questionId]]
  temp = temp.dropna(subset=['discursiva.respostaCandidato'])
  expected_answers = [questions['discursiva.expectativaDeResposta'][questionId]]
  #filtered_tokens = [token.text for token in expected_answers if not token.is_stop]
  #expected_answers = nlp(' '.join(filtered_tokens))

  for index, row in temp.iterrows():
    #print(row)
    
    student_answer = [row['discursiva.respostaCandidato']]
    #student_answer = [token.text for token in student_answer if not token.is_stop]
    #student_answer_filtered_tokens = nlp(' '.join(student_answer))
    #print(expected_answers,student_answer)
    similarity = jaccard_coefficient(expected_answers, student_answer)
    print(similarity)
    temp.loc[index, 'jaccard_similarity'] = similarity
    print(f'{row.nomeCandidato} => {row.notaQuestao} => {similarity}')

  jaccard_similarity = pd.concat([jaccard_similarity, temp])


print(jaccard_similarity['jaccard_similarity'])

['polimorf', 'é', 'uma', 'característ', 'das', 'linguagens', 'orient', 'a', 'objet', 'que', 'permit', 'que', 'um', 'program', 'process', 'objet', 'de', 'form', 'diferent', 'a', 'depend', 'dos', 'tip', 'de', 'dad', 'ou', 'class', 'os', 'princip', 'tip', 'de', 'polimorf', 'sã', 'estát', 'temp', 'de', 'compil', 'e', 'o', 'dinâm', 'polimorf', 'estát', 'possibilit', 'execut', 'oper', 'diferent', 'de', 'um', 'mesm', 'métod', 'a', 'depend', 'dos', 'parâmetr', 'pass', 'é', 'cham', 'de', 'estát', 'pois', 'o', 'compil', 'sab', 'em', 'temp', 'de', 'compil', 'exat', 'o', 'métod', 'que', 'ser', 'cham', 'o', 'polimorf', 'dinâm', 'ocorr', 'quand', 'há', 'sobrescrit', 'de', 'métod', 'em', 'class', 'deriv', 'com', 'isso', 'a', 'depend', 'da', 'instânc', 'oper', 'diferent', 'pod', 'ser', 'execut', 'exempl', 'polimorf', 'estát', 'public', 'class', 'operations', 'public', 'void', 'operat', 'int', 'op', 'system', 'out', 'println', 'quot', 'operating', 'int', 'quot', 'public', 'void', 'operat', 'string', 'o